In [23]:
from insolAPI.WebAPI import API
import simplejson as json
import pandas as pd
import pendulum as pdl
from tqdm import tqdm
import warnings
from tabulate import tabulate



installation_path = "C:/Users/Insolight/Desktop/InsolReports/Installations/"
with open(installation_path+'/local.json') as f:
    local_data = json.load(f)


class Color:
    RED = '\033[91m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    BLUE = '\033[94m'
    RESET = '\033[0m'


install_number = 24

instal_dict = {
    "Agroscope Series": 24,
    "Agroscope Berries": 25,
    "Bioschmid": 23,
    "Etchelecu": 26
}

api = API(local_data['API_user'], local_data['API_pwd'], dev_space=False)


time_args = dict(
    start=pdl.now().subtract(hours=2, minutes=30).to_datetime_string(),
    stop=pdl.now().subtract(hours=2).to_datetime_string(),
    timezone="UTC",
)

list_sensor = ["PAR", "IRRAD", "GII", "DNI", "DHI", "TEMP", "HUMI", "RAIN", "RAIN_TYPE", "RAIN_ACCUMULATED", "WIND", "WIND_DIR", "PRESSURE", "CURRENT", "VOLTAGE", "VIRTUAL", "LEAF_TEMP",
                     "SOIL_MOISTURE", "SOIL_PH", "RAW", "RAW2", "POS", "POS_ANGLE", "HAMAMATSU", "FEEDBACK_CELL"]

In [24]:
dict_instal_logs = {}
dict_sensor_channel_id = {}
diff_logs = {}
logs_joined = {}
dict_channel_id = {}
dict_missing_sensors = {}


#gather all the logs for each installation
for instal in tqdm(instal_dict):
    list_channel_id = []
    list_sensor_logging = []

    logs_joined[instal] = {}
    dict_sensor_channel_id[instal] = {}
    sensor_number = 0
    for sensor_type in api.SensorsTypes:
        try:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", category=UserWarning)

                logs_joined[instal][list_sensor[sensor_number]] = api.get_sensor_channels_logs_joined(**time_args, sensor_type=sensor_type, install=instal_dict[instal])
                list_sensor_logging.extend(logs_joined[instal][list_sensor[sensor_number]].sensor_channel_id.unique())
        except:
            pass
        try:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", category=UserWarning)

                dict_sensor_channel_id[instal][list_sensor[sensor_number]] = api.get_sensor_channels(sensor_type=sensor_type, install=instal_dict[instal])
                list_channel_id.extend(dict_sensor_channel_id[instal][list_sensor[sensor_number]][dict_sensor_channel_id[instal][list_sensor[sensor_number]]['deleted_at'].isnull()].index.to_list())
        except:
            pass
        sensor_number += 1
    dict_instal_logs[instal] = list_sensor_logging
    dict_channel_id[instal] = list_channel_id


#check if there are missing sensors
for instal in instal_dict:
    bool_missing_sensor = False
    diff_logs[instal] = list(set(dict_channel_id[instal]) - set(dict_instal_logs[instal]))
    print(f"{instal}:")
    for sensor_id in diff_logs[instal]:
        for sensor in list_sensor:
            try:
                dict_missing_sensors[sensor_id] = [instal, dict_sensor_channel_id[instal][sensor].loc[sensor_id].gateway_id, dict_sensor_channel_id[instal][sensor].loc[sensor_id].sensor_name, dict_sensor_channel_id[instal][sensor].loc[sensor_id].channel_name]
                print(Color.RED + f"{sensor_id}: {dict_missing_sensors[sensor_id]}" + Color.RESET)
                bool_missing_sensor = True
            except:
                pass
            sensor_number += 1
    if not bool_missing_sensor:
        print(Color.GREEN + "No missing sensors\n" + Color.RESET)



df_missing_sensors = pd.DataFrame.from_dict(dict_missing_sensors, orient='index', columns=['installation', 'gateway_id', 'sensor_name', 'channel_name'])
df_missing_sensors = df_missing_sensors.drop_duplicates(subset=['sensor_name'])
df_missing_sensors


100%|██████████| 4/4 [00:27<00:00,  6.96s/it]

Agroscope Series:
768: ['Agroscope Series', 29, 'leaf_temp_FS1', 'temp_max']
769: ['Agroscope Series', 29, 'leaf_temp_FS1', 'temp_raw']
770: ['Agroscope Series', 29, 'leaf_temp_FS1', 'temp_sensor']
739: ['Agroscope Series', 29, 'rain', 'rainfall']
740: ['Agroscope Series', 29, 'rain', 'computed_rate']
741: ['Agroscope Series', 29, 'rain', 'rainfall_acc']
742: ['Agroscope Series', 29, 'rain', 'precipitation_type']
766: ['Agroscope Series', 29, 'leaf_temp_FS1', 'temp_avg']
767: ['Agroscope Series', 29, 'leaf_temp_FS1', 'temp_min']
Agroscope Berries:
No missing sensors

Bioschmid:
No missing sensors

Etchelecu:
No missing sensors



,installation,gateway_id,sensor_name,channel_name
768,Agroscope Series,29,leaf_temp_FS1,temp_max
739,Agroscope Series,29,rain,rainfall


In [35]:
columns = pd.MultiIndex.from_tuples([('', 'Installation'), ('Sensor', 'ID'),('Sensor', 'Name'), ('weather', 'snow'), ('weather', 'wind')])
df_report = pd.DataFrame(columns=columns)

#fill the report with the missing sensors dict, with one line per installation, and a list of the missing sensors
for instal in instal_dict:
    df_report.loc[instal, ('', 'Installation')] = instal
    df_report.loc[instal, ('Sensor', 'ID')] = df_missing_sensors[df_missing_sensors['installation'] == instal].gateway_id.to_list()
    df_report.loc[instal, ('Sensor', 'Name')] = df_missing_sensors[df_missing_sensors['installation'] == instal].sensor_name.to_list()
    df_report.loc[instal, ('weather', 'snow')] = ["2023-12-11 00:00:00+00:00", "2023-12-11 00:00:00+00:00", "2023-12-11 00:00:00+00:00", "2023-12-11 00:00:00+00:00", "2023-12-11 00:00:00+00:00", "2023-12-11 00:00:00+00:00", "2023-12-11 00:00:00+00:00", "2023-12-11 00:00:00+00:00"]
    df_report.loc[instal, ('weather', 'wind')] = False
#reset index
df_report = df_report.reset_index(drop=True)

In [46]:
#make a new df, which has the same content of the df_report, but remove all the lists, and replace them by a string, with a \n between each element
df_report_string = df_report.copy()
def list_to_string(lst):
    try:
        return '\n'.join(map(str, lst))
    except:
        return lst

for col in df_report_string.columns:
    print(col)
    if col[1] == 'Installation':
        continue
    df_report_string[col] = df_report_string[col].apply(list_to_string)

('', 'Installation')
('Sensor', 'ID')
('Sensor', 'Name')
('weather', 'snow')
('weather', 'wind')


In [47]:
df_report_string

Sensor                        
        Installation      ID                 Name   
0   Agroscope Series  29\n29  leaf_temp_FS1\nrain  \
1  Agroscope Berries                                
2          Bioschmid                                
3          Etchelecu                                

                                             weather         
                                                snow   wind  
0  2023-12-11 00:00:00+00:00\n2023-12-11 00:00:00...  False  
1  2023-12-11 00:00:00+00:00\n2023-12-11 00:00:00...  False  
2  2023-12-11 00:00:00+00:00\n2023-12-11 00:00:00...  False  
3  2023-12-11 00:00:00+00:00\n2023-12-11 00:00:00...  False

In [36]:
df_report

Sensor                          
        Installation        ID                   Name   
0   Agroscope Series  [29, 29]  [leaf_temp_FS1, rain]  \
1  Agroscope Berries        []                     []   
2          Bioschmid        []                     []   
3          Etchelecu        []                     []   

                                             weather         
                                                snow   wind  
0  [2023-12-11 00:00:00+00:00, 2023-12-11 00:00:0...  False  
1  [2023-12-11 00:00:00+00:00, 2023-12-11 00:00:0...  False  
2  [2023-12-11 00:00:00+00:00, 2023-12-11 00:00:0...  False  
3  [2023-12-11 00:00:00+00:00, 2023-12-11 00:00:0...  False

In [48]:
print(tabulate(df_report_string, headers='keys', tablefmt='grid', showindex=False))

+------------------------+--------------------+----------------------+---------------------------+-----------------------+
| ('', 'Installation')   | ('Sensor', 'ID')   | ('Sensor', 'Name')   | ('weather', 'snow')       | ('weather', 'wind')   |
+========================+====================+======================+===========================+=======================+
| Agroscope Series       | 29                 | leaf_temp_FS1        | 2023-12-11 00:00:00+00:00 | False                 |
|                        | 29                 | rain                 | 2023-12-11 00:00:00+00:00 |                       |
|                        |                    |                      | 2023-12-11 00:00:00+00:00 |                       |
|                        |                    |                      | 2023-12-11 00:00:00+00:00 |                       |
|                        |                    |                      | 2023-12-11 00:00:00+00:00 |                       |
|               

In [50]:
instal_dict = {
    "Agroscope Series": 24,
    "Agroscope Berries": 25,
    "Bioschmid": 23,
    "Etchelecu": 26
}

for loc in instal_dict:
    print(loc)


Agroscope Series
Agroscope Berries
Bioschmid
Etchelecu
